In [1]:
import time
import matplotlib.pyplot as plt
from scipy import linalg
import sympy as sym
import numpy as np

## Dynamic model

# Specify the physical parameters:
## mb: mass of body
## Jxb, Jyb, Jzb are the inertia values for the body 
## mw: mass of reaction wheels 
## Jxw, Jyw, Jzw are the inertia values for the reaction wheels
## lw is a dimention parameter for the wheel itself


In [2]:
mb = 6.
Jxb = 10.
Jyb = 10.
Jzb = 16.

mw = 1.
Jxw = 0.075
Jyw = 0.075
Jzw = 0.125
lw = 1.1

Derive the equations of motion:

In [3]:
# roll, pitch, yaw angles
phi, theta, psi = sym.symbols('phi, theta, psi')

# angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# resultant torques
lt = sym.nsimplify(lw) * sym.sqrt(2) / 2
T1 = - tau_1 * sym.Matrix([[lt], [0], [lt]])
T2 = - tau_2 * sym.Matrix([[-lt], [0], [lt]])
T3 = - tau_3 * sym.Matrix([[0], [lt], [lt]])
T4 = - tau_4 * sym.Matrix([[0], [-lt], [lt]])
T = T1 + T2 + T3 + T4

# parameters
Jx = sym.nsimplify(Jxb + 4 * mw * lw**2)
Jy = sym.nsimplify(Jyb + 4 * mw * lw**2)
Jz = sym.nsimplify(Jzb + 4 * mw * lw**2)

# rotation matrices
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry * Rz).T * ex, Rz.T * ey, ez).inv(), full=True)

# euler's equations
euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                   [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                   [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

# equations of motion
f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)
print(T)

Matrix([[-11*sqrt(2)*tau_1/20 + 11*sqrt(2)*tau_2/20], [-11*sqrt(2)*tau_3/20 + 11*sqrt(2)*tau_4/20], [-11*sqrt(2)*tau_1/20 - 11*sqrt(2)*tau_2/20 - 11*sqrt(2)*tau_3/20 - 11*sqrt(2)*tau_4/20]])


The equations of motion have this form:

$$\begin{bmatrix} \dot{\phi} \\ \dot{\theta} \\ \dot{\psi} \\ \dot{w_x} \\ \dot{w_y} \\ \dot{w_z} \end{bmatrix} = f\left(\phi, \theta, \psi, w_x, w_y, w_z, \tau_1, \tau_2, \tau_3, \tau_4\right)$$

Here is the function $f$:

In [4]:
f

Matrix([
[                        (w_x*cos(psi) - w_y*sin(psi))/cos(theta)],
[                                     w_x*sin(psi) + w_y*cos(psi)],
[        -w_x*cos(psi)*tan(theta) + w_y*sin(psi)*tan(theta) + w_z],
[-55*sqrt(2)*tau_1/1484 + 55*sqrt(2)*tau_2/1484 - 150*w_y*w_z/371],
[-55*sqrt(2)*tau_3/1484 + 55*sqrt(2)*tau_4/1484 + 150*w_x*w_z/371],
[                -55*sqrt(2)*(tau_1 + tau_2 + tau_3 + tau_4)/2084]])

# Making "f" an executable function
## Lambdify is setup in a way that values can be easily substituted in for the variables

In [5]:
f_num = sym.lambdify((phi,theta,psi,w_x,w_y,w_z,tau_1,tau_2,tau_3,tau_4),f)

# Defining a set of Equilibrium Values for the states so that f() == 0
## These values are specifically chosen so that the control system may converge to those desired values. And in this case, we want the spacecraft to maintain its attitude given external disturbances. 

In [6]:
phi_e = 0
theta_e = 0
psi_e = 0
wx_e = 0
wy_e = 0
wz_e = 0
tau1_e = 0
tau2_e = 0
tau3_e = 0
tau4_e = 0

## 1) Making a function that subsitutes equilibrium values into the equations of motion 
## 2) Verify the equilibrium values make f() == 0
## 3) Print out the f matrix and make sure it returns a 6 by 1 column matrix

In [7]:
feq = f_num(phi_e,theta_e,psi_e,wx_e,wy_e,wz_e,tau1_e,tau2_e,tau3_e,tau4_e)
feq

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [-0.]])

# Linearizing the equations of motion 

## x: state vector (variables we are interested in)
## $\dot{x}$: time rate change of state
## u: inputs vector (actuators in the system that we control)
## A matrix: Relates the states to the time rate change of state
## B matrix: Relates the inputs to the time rate change of state

## 1) $\dot{x}$ = Ax + Bu. To get the A matrix, we apply a Jacobian function, which takes the partial derivatives of all of the equations of motion with respect to each state variable. In this case, we call the Jacobian on our symbolic function "f".


## 2) Only ($\phi$, $\theta$, $\psi$, $w_x$, $w_y$, $w_z$) will be accounted for in the A matrix since the other four ($\tau_1$, $\tau_2$, $\tau_3$, and $\tau_4$) are the inputs "u". So those are accounted for in the B matrix.


In [8]:
states = [phi,theta, psi, w_x, w_y, w_z]
A_sym = f.jacobian(states)

inputs = [tau_1, tau_2, tau_3, tau_4]
B_sym = f.jacobian(inputs)

## We want to linearize about our equilibirium values, so we need to sub in the equilibrium values into the symbolic A and B matrices.

In [9]:
A_num = sym.lambdify(states,A_sym)
B_num = sym.lambdify(inputs,B_sym)

A = A_num(phi_e,theta_e,psi_e,wx_e,wy_e,wz_e)
B = B_num(tau1_e, tau2_e, tau3_e, tau4_e)

# Checking for controllability
## $\dot{x}$ = Ax + Bu where u = -Kx
## K is a constant matrix that we choose that tells us what our inputs should be, given our state.

## First, we create the controllability matrix, as shown below

## We can check if the controllability matrix is of full rank, meaning that the system is controllable

In [10]:

#* Number of states
n = len(B)

#* Initialize the Controllability Matrix
W = B

#* Create the Controllability Matrix
for i in range(1,n):
    new_mat = np.linalg.matrix_power(A, i) @ B
    W = np.block([W,new_mat])

#* Make sure that the rank of the matrix is equal to the number of states
if (np.linalg.matrix_rank(W) == n):
    print("FULL RANK")
else:
    print("RANK DEFICIENT") 

FULL RANK


# LQR Design (Linear Quadratic Regulator)

## LQR is useful for optimal control - choosing a K matrix to stabilize the system without too much effort

## Q and R matrices need to be implemented where Q can be seen as a "penalty" matrix that tells us how bad the penalty is if the states are not at where we want them to be. In other words, by looking at the diagonals of this matrix, we could assign values cooresponding to the different states we are interested in. We start with an identity matrix I, with 1s in the diagonal. If we are interested in making the state "theta" or "theta dot" go to our desired state faster, we could put 10 or 100 in those slots. 

## The R matrix tells us how bad the penalty is for using our actuators.


In [11]:
## Designing a Q matrix 
Q_diag = np.array([1,1,1,10,10,10]) #row, pitch, yall, wx, wy, wz
Q = np.diag(Q_diag)

## Designing a R matrix
R_diag = np.array([0.1,0.1,0.1,0.1]) #tau1, tau2, tau3, tau4
R = np.diag(R_diag)

#* Making a LQR function that outputs k
#* We solve the Ricatti equation for optimal controller gains
def lqr(A,B,Q,R):
    P = linalg.solve_continuous_are(A,B,Q,R)
    K = linalg.inv(R) @ B.T @ P
    return K
K = lqr(A,B,Q,R)

##* testing to see if the A - BK has all negative eigenvalues 
F = A - B@K
eig = np.linalg.eigvals(F)

print(eig.real < 0)


[ True  True  True  True  True  True]


## Sensor model

Symbolic variables for right ascension $\alpha$ and declination $\delta$ of each star:

In [12]:
alpha, delta = sym.symbols('alpha, delta')

Specify the physical parameters:

In [13]:
# Scope radius
r = 0.8 / 2.1

Derive the sensor model:

In [14]:
# Position of star in space frame
p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                              [sym.sin(alpha) * sym.cos(delta)],
                              [sym.sin(delta)]])

# Orientation of body frame in space frame
R_body_in_space = Rx * Ry * Rz

# Position of star in body frame (assuming origin of body and space frames are the same)
p_star_in_body = R_body_in_space.T * p_star_in_space

# Position of star in image frame
p_star_in_image = (1 / sym.nsimplify(r)) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                                       [p_star_in_body[2] / p_star_in_body[0]]])

# Sensor model for each star
g = sym.simplify(p_star_in_image, full=True) 

The sensor model has this form for each star:

$$\zeta = g(\phi, \theta, \psi, \alpha, \delta)$$

Here is the function $g$:

In [15]:
g

Matrix([
[21*((sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))*sin(delta) - (sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi))*sin(alpha)*cos(delta) - sin(psi)*cos(alpha)*cos(delta)*cos(theta))/(8*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*sin(delta) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*sin(alpha)*cos(delta) + cos(alpha)*cos(delta)*cos(psi)*cos(theta)))],
[                                                                      21*(-sin(alpha)*sin(phi)*cos(delta)*cos(theta) + sin(delta)*cos(phi)*cos(theta) + sin(theta)*cos(alpha)*cos(delta))/(8*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*sin(delta) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*sin(alpha)*cos(delta) + cos(alpha)*cos(delta)*cos(psi)*cos(theta)))]])

## g is a function of the $\alpha$,$\delta$, and other state variables that give us the pixel positions for each star

In [16]:
# making G a combination of multiple stars, in this case, we are doing 3 stars
alpha1 = 0
delta1 = 0
alpha2 = 0
delta2 = 0.2
alpha3 = 0
delta3 = 0

G = np.block([[g.subs([(alpha,alpha1),(delta,delta1)])],[g.subs([(alpha,alpha2),(delta,delta2)])],[g.subs([(alpha,alpha3),(delta,delta3)])]])
G = sym.Matrix(G) #turning an array into a symbolic matrix expression
G


Matrix([
[                                                                                                                                                                                                                                           -21*sin(psi)/(8*cos(psi))],
[                                                                                                                                                                                                                               21*sin(theta)/(8*cos(psi)*cos(theta))],
[21*(0.198669330795061*sin(phi)*cos(psi) + 0.198669330795061*sin(psi)*sin(theta)*cos(phi) - 0.980066577841242*sin(psi)*cos(theta))/(8*(0.198669330795061*sin(phi)*sin(psi) - 0.198669330795061*sin(theta)*cos(phi)*cos(psi) + 0.980066577841242*cos(psi)*cos(theta)))],
[                                                        21*(0.980066577841242*sin(theta) + 0.198669330795061*cos(phi)*cos(theta))/(8*(0.198669330795061*sin(phi)*sin(psi) - 0.198669330795061*sin(thet

## Next step is to find the equilibrium state for G. Same thing as before, we need to use Lambdify so that we could sub in the equilibrium values of phi, theta, and psi


In [17]:
G_num = sym.lambdify((phi, theta, psi), G)
Geq = G_num(phi_e, theta_e, psi_e)
Geq.tolist()

[[-0.0], [0.0], [0.0], [0.5321138432102644], [-0.0], [0.0]]

## Now, we linearize the system by taking the Jacobian of the equations of motion for the rate of change of our outputs with respect to our states

In [18]:
C_num = sym.lambdify((phi, theta, psi), G.jacobian([phi, theta, psi, w_x, w_y, w_z]))
C = C_num(phi_e,theta_e,psi_e)
C

array([[ 0.        ,  0.        , -2.625     ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  2.625     ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.53211384,  0.        , -2.625     ,  0.        ,  0.        ,
         0.        ],
       [-0.        ,  2.73286482,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        , -2.625     ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  2.625     ,  0.        ,  0.        ,  0.        ,
         0.        ]])